In [1]:
using BenchmarkTools
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

In [2]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [3]:
x = Lar.Struct([square])

LinearAlgebraicRepresentation.Struct([([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])], [[0.0; 0.0], [1.0; 1.0]], "7377560575363784694", 2, "feature")

## Vecchia funzione

In [4]:
function embedStruct(n::Int)
	function embedStruct0(self::Lar.Struct,suffix::String="New")
		if n==0
			return self, length(self.box[1])
		end
		cloned = Lar.Struct()
		cloned.box = [ [self.body[i].box[1];zeros(dimadd)],
						[self.body[i].box[2];zeros(dimadd)] ]
		cloned.name = self.name*suffix
		cloned.category = self.category
		cloned.dim = self.dim+n
		cloned = Lar.embedTraversal(cloned,self,n,suffix)
		return cloned
	end
	return embedStruct0
end

embedStruct (generic function with 1 method)

In [ ]:
@btime embedStruct(1)(x)

In [ ]:
@benchmark embedStruct(1)(x)

## Modifiche

In [16]:
function embedStruct(n)
	function embedStruct0(self, suffix)
		if n==0
			return self, length(self.box[1])
		end
		cloned = Lar.Struct()
		cloned.box = hcat((self.box,[fill([0],n),fill([0],n)]))
		cloned.name = self.name*suffix
		cloned.category = self.category
		cloned.dim = self.dim+n
		cloned = Lar.embedTraversal(cloned,self,n,suffix)
		return cloned
	end
	return embedStruct0
end

embedStruc (generic function with 1 method)

In [17]:
@btime embedStruct(1)(x,"-New")

  11.300 μs (33 allocations: 2.45 KiB)


LinearAlgebraicRepresentation.Struct(Any[[([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])]], [([[0.0; 0.0], [1.0; 1.0]], [[[0]], [[0]]])], "7377560575363784694-New", 3, "feature")

In [6]:
@benchmark embedStruct(1)(x,"-New")

BenchmarkTools.Trial: 
  memory estimate:  2.45 KiB
  allocs estimate:  33
  --------------
  minimum time:     11.400 μs (0.00% GC)
  median time:      12.201 μs (0.00% GC)
  mean time:        16.980 μs (0.00% GC)
  maximum time:     438.400 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [7]:
@code_typed embedStruct(1)(x,"-New")

CodeInfo(
1 ── %1   = Core.getfield(#self#, :n)::Int64
│    %2   = (%1 === 0)::Bool
└───        goto #3 if not %2
2 ── %4   = Base.getfield(self, :box)::Any
│    %5   = Base.getindex(%4, 1)::Any
│    %6   = Main.length(%5)::Any
│    %7   = Core.tuple(self, %6)::Tuple{LinearAlgebraicRepresentation.Struct,Any}
└───        return %7
3 ── %9   = Base.getproperty(Main.Lar, :Struct)::Any
│    %10  = (%9)()::Any
│    %11  = Base.getfield(self, :box)::Any
│    %12  = $(Expr(:foreigncall, :(:jl_alloc_array_1d), Array{Int64,1}, svec(Any, Int64), 0, :(:ccall), Array{Int64,1}, 1, 1))::Array{Int64,1}
│           Base.arraysize(%12, 1)::Int64
└───        goto #5 if not true
4 ──        Base.arrayset(false, %12, 0, 1)::Array{Int64,1}
5 ┄─        goto #6
6 ── %17  = Core.getfield(#self#, :n)::Int64
│    %18  = $(Expr(:foreigncall, :(:jl_alloc_array_1d), Array{Array{Int64,1},1}, svec(Any, Int64), 0, :(:ccall), Array{Array{Int64,1},1}, :(%17), :(%17)))::Array{Array{Int64,1},1}
│    %19  = Base.arraysize

In [ ]:
@benchmark embedStruct(1)(x,"-New")